In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import random

import scipy.stats as stats

In [2]:
df = pd.read_csv('../data/kandidater_data.csv').dropna()

In [3]:
# gender dummy question
df['26'] = df["Gender"].map(lambda x: -2 if x == "M" else 2) # random.choices([2, -2], weights=[0.5, 0.5])[0]

# age dummy question
df['Age'] = 2022 - df.Birthdate.str[:4].astype(int)
df['27'] = df['Age'].map(lambda x: 0 if x <= 30 else 2)

# block dummy question
blok_dict = {
    'A': 1,
    'Ø': 1,
    'F': 1,
    'Å': 1,
    'Q': 1,
    'V': 0,
    'C': 0,
    'O': 0,
    'I': 0,
    'Æ': 0,
    'D': 0,
    # ikke så rød-blå
    'B': 1,
    'M': 0,
    'K': 0
}

df['Block'] = df.CurrentPartyCode.map(blok_dict)
df['28'] = df.Block.apply(lambda x: 0 if x==1 else 2)

# education dummy variable
edu_map = {
    'Grundskole': 1,
    '10. klasse': 2,
    'Erhvervsuddannelse': 4,
    'Gymnasial uddannelse': 4,
    'Seminaruddannelse': 6,
    'Bachelor-/diplomuddannelse': 6,
    'Kandidat-/masteruddannelse': 7,
    'Ph.d-uddannelse': 8
}
df['EducationLevel'] = df.Education.map(edu_map)
df['29'] = df.EducationLevel.apply(lambda x: -2 if x >= 6 else 2)

# Profession
political_professions = {
    'Folketingsmedlem' : True,
     'Minister' : True,
     'Ikke oplyst' : False,
     'Offentligt ansat' : False,
     'Byråds- eller regionsrådsmedlem' : True,
     'Privat ansat/selvstændig' : False,
     'Studerende' : False,
     'Leder' : False,
     'Ordfører for et parti' : True,
     'Udenfor arbejdsmarkedet' : False,
     'Partileder' : True,
     'Europa-Parlamentariker' : True,
}

df["IsPolitical"] = df["Profession"].map(political_professions)
df['30'] = df.IsPolitical.apply(lambda x: -2 if x else 2)

In [4]:
def propensity(df, y, onehotvars=[], minmaxvars=[]):
    clf = LogisticRegression()

    ct = ColumnTransformer([
        ('scaler', MinMaxScaler(), minmaxvars),
        ('one hot', OneHotEncoder(), onehotvars)
    ])

    pipe = Pipeline([
        ('column transformer', ct), 
        ('logistic reg', clf)
    ])
    
    pipe.fit(df, y)
    return pipe.predict_proba(df)[:, 1]

In [5]:
def match(df1, df2):
    x = df1.propensity.to_numpy().reshape(-1, 1)
    y = df2.propensity.to_numpy().reshape(-1, 1)
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(x)
    _, idx = nn.kneighbors(y)
    return df1.iloc[idx.reshape(-1)], df2

def match2(df1, df2):
    x = df2.propensity.to_numpy().reshape(-1, 1)
    y = df1.propensity.to_numpy().reshape(-1, 1)
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(x)
    _, idx = nn.kneighbors(y)
    return df2.iloc[idx.reshape(-1)], df1

In [6]:
params = {
    'onehotvars': ['Profession', 'Education', 'Gender'],
    'minmaxvars': ['Age']
}

y = df['28'].to_numpy()
df['propensity'] = propensity(df, y, **params)

In [7]:
mask = df['28'] > 0
df1 = df[mask]
df2 = df[~mask]
df1, df2 = match(df1, df2)

In [8]:
_, p_values = stats.ttest_rel(df1[[str(i) for i in range(1, 31)]], df2[[str(i) for i in range(1, 31)]])

<ipython-input-8-78183864cdcb>:1: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  _, p_values = stats.ttest_rel(df1[[str(i) for i in range(1, 31)]], df2[[str(i) for i in range(1, 31)]])


In [9]:
np.where(p_values <= 0.002)[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 19, 20, 21, 22, 23, 24, 26, 27], dtype=int64)

In [10]:
plt.style.use('ggplot')

In [13]:
fig, ax = plt.subplots(figsize=(6,10))
y = np.arange(25)
x1 = np.array([df1[str(i)].mean() for i in range(1, 26)])
x2 = np.array([df2[str(i)].mean() for i in range(1, 26)])

X = np.vstack((x1, x2)).T
idx = np.argsort(-np.mean(X, axis=1))
# idx = np.argsort(np.abs(x1 - x2))
x1 = x1[idx]
x2 = x2[idx]

for n in y:
    a = min(x1[n], x2[n])
    b = max(x1[n], x2[n])
    ax.hlines(n, a, b, color='grey')

ax.scatter(x1, y, label='rød')
ax.scatter(x2, y, label='blå')

ax.set_yticks(y)

ax.set_title('Difference between right/left-wing mean opinion')
ax.set_xlabel('Mean question answer')
ax.set_ylabel('Question number')

ax.set_yticklabels(y[idx])
ax.legend()